In [14]:
import numpy as np
from pathlib import Path
import nibabel as nib
from typing import List
import concurrent.futures
from tqdm import tqdm

In [15]:
brats2021_label_path: Path = Path(
    "/home/stud/strasser/archive/brats2021/"
)
brats2019_train_path: Path = Path(
    "/home/stud/strasser/archive/brats2019/MICCAI_BraTS_2019_Data_Training"
)

In [16]:
brats2021_label_nii_gz_file_paths: List[Path] = []

for folder in tqdm(list(brats2021_label_path.glob("*"))):
    brats2021_label_nii_gz_file_paths += list(folder.glob("*t2.nii.gz"))


brats2021_label_nii_gz_file_paths[:10]

  0%|          | 0/1251 [00:00<?, ?it/s]

100%|██████████| 1251/1251 [00:01<00:00, 1210.70it/s]


[PosixPath('/home/stud/strasser/archive/brats2021/BraTS2021_01196/BraTS2021_01196_t2.nii.gz'),
 PosixPath('/home/stud/strasser/archive/brats2021/BraTS2021_01002/BraTS2021_01002_t2.nii.gz'),
 PosixPath('/home/stud/strasser/archive/brats2021/BraTS2021_01075/BraTS2021_01075_t2.nii.gz'),
 PosixPath('/home/stud/strasser/archive/brats2021/BraTS2021_01178/BraTS2021_01178_t2.nii.gz'),
 PosixPath('/home/stud/strasser/archive/brats2021/BraTS2021_00767/BraTS2021_00767_t2.nii.gz'),
 PosixPath('/home/stud/strasser/archive/brats2021/BraTS2021_00684/BraTS2021_00684_t2.nii.gz'),
 PosixPath('/home/stud/strasser/archive/brats2021/BraTS2021_00789/BraTS2021_00789_t2.nii.gz'),
 PosixPath('/home/stud/strasser/archive/brats2021/BraTS2021_01432/BraTS2021_01432_t2.nii.gz'),
 PosixPath('/home/stud/strasser/archive/brats2021/BraTS2021_01445/BraTS2021_01445_t2.nii.gz'),
 PosixPath('/home/stud/strasser/archive/brats2021/BraTS2021_01548/BraTS2021_01548_t2.nii.gz')]

In [17]:
brats_2019_hgg = brats2019_train_path / "HGG"
brats_2019_lgg = brats2019_train_path / "LGG"

hgg_paths = [
    folder
    for instance in brats_2019_hgg.iterdir()
    if instance.is_dir()
    for folder in list(instance.glob("*t2.nii"))
]
lgg_paths = [
    folder
    for instance in brats_2019_lgg.iterdir()
    if instance.is_dir()
    for folder in list(instance.glob("*t2.nii"))
]

brats2019_segmentation_mask_file_paths: List[Path] = hgg_paths + lgg_paths
brats2019_segmentation_mask_file_paths[:10]

[PosixPath('/home/stud/strasser/archive/brats2019/MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_CBICA_BHV_1/BraTS19_CBICA_BHV_1_t2.nii'),
 PosixPath('/home/stud/strasser/archive/brats2019/MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_CBICA_BGN_1/BraTS19_CBICA_BGN_1_t2.nii'),
 PosixPath('/home/stud/strasser/archive/brats2019/MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_TCIA01_425_1/BraTS19_TCIA01_425_1_t2.nii'),
 PosixPath('/home/stud/strasser/archive/brats2019/MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_TCIA02_168_1/BraTS19_TCIA02_168_1_t2.nii'),
 PosixPath('/home/stud/strasser/archive/brats2019/MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_TCIA03_121_1/BraTS19_TCIA03_121_1_t2.nii'),
 PosixPath('/home/stud/strasser/archive/brats2019/MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_TCIA04_328_1/BraTS19_TCIA04_328_1_t2.nii'),
 PosixPath('/home/stud/strasser/archive/brats2019/MICCAI_BraTS_2019_Data_Training/HGG/BraTS19_TCIA05_396_1/BraTS19_TCIA05_396_1_t2.nii'),
 PosixPath('/home/stud/strasser/archiv

In [18]:
# now check for one file of 2019, if its segmentation mask occurs in 2021

brats2019_segmentation_path = brats2019_segmentation_mask_file_paths[10]
brats2019_segmentation_image = nib.load(brats2019_segmentation_path)

# Numpy-Array aus der .nii-Datei extrahieren
brats2019_segmentation_np = brats2019_segmentation_image.get_fdata()
brats2019_segmentation_np.shape

(240, 240, 155)

In [7]:
# # Naive without multitasking

# for brats2018_fp in tqdm(brats2018_segmentation_mask_file_paths):
#     brats2018_segmentation_nparr = nib.load(brats2018_fp).get_fdata()

#     for brats2021_nii_gz_path in brats2021_label_nii_gz_file_paths:
#         brats2021_segmentation_image = nib.load(brats2021_nii_gz_path)
#         brats2021_segmentation_np = brats2021_segmentation_image.get_fdata()

#         # check for equality
#         if np.array_equal(brats2018_segmentation_nparr, brats2021_segmentation_np):
#             raise ValueError(f"Duplicate in segmentation mask for file {brats2021_nii_gz_path}")

# print("no overlap")

In [19]:
print("brats 2019: ", len(brats2019_segmentation_mask_file_paths))
print("brats 2021: ", len(brats2021_label_nii_gz_file_paths))

print(
    "product: ",
    len(brats2019_segmentation_mask_file_paths)
    * len(brats2021_label_nii_gz_file_paths),
)

brats 2019:  335
brats 2021:  1251
product:  419085


In [9]:
def check_segmentation_equality(brats2019_fp, brats2021_nii_gz_path):
    brats2019_segmentation_nparr = nib.load(brats2019_fp).get_fdata()
    brats2021_segmentation_image = nib.load(brats2021_nii_gz_path)
    brats2021_segmentation_np = brats2021_segmentation_image.get_fdata()

    if np.array_equal(brats2019_segmentation_nparr, brats2021_segmentation_np):
        raise ValueError(
            f"Duplicate in segmentation mask for file {brats2021_nii_gz_path}"
        )


NUM_WORKERS = 16

with concurrent.futures.ThreadPoolExecutor(max_workers=NUM_WORKERS) as executor:
    futures = []
    total = 0

    for brats2019_fp in brats2019_segmentation_mask_file_paths:
        for brats2021_nii_gz_path in brats2021_label_nii_gz_file_paths:
            total += 1
            futures.append(
                executor.submit(
                    check_segmentation_equality, brats2019_fp, brats2021_nii_gz_path
                )
            )

    done = 0
    for future in concurrent.futures.as_completed(futures):
        future.result()

        done += 1
        print(f"checked {done} / {total}", end="\r")

print("no overlap")